# sPlot with simple Gaussian and polynomial PDFs

Fit a pseudo data missing mass distribution to produce sWeights then use them to plot Signal weighted distribution for other variables.

Note the weights are stored in a Weights object which then can be loaded into other fits. [Weights.h](https://github.com/dglazier/brufit/blob/master/core/Weights.h)

It is also possible to merge with the data tree in ROOT (similarily in pyROOT) :

    Weights wgts;
    wgts.LoadSaved("filename.root","HSsWeights");
    wgts.AddToTree(tree);

Load brufit using ROOT python bindings and initialise jsroot for drawing histograms.

In [ ]:
import ROOT
ROOT.gROOT.ProcessLine(".x $BRUFIT/macros/LoadBru.C")
%jsroot

First you will need to generate some data. This is done with a ROOT macro generating random numbers from TF1 functions. It can be executed from the following notebook GenerateData
Link to [Generate Data](GenerateData.ipynb)

Create the sPlot fit manager and set the ouput directory for fit results, plots and weights

In [ ]:
splot = ROOT.sPlot()
splot.SetUp().SetOutDir("outSplot")

Define the fit variable as Mmiss which is the name of a branch in the tree and set the fit range to 0-10

In [ ]:
splot.SetUp().LoadVariable("Mmiss[0,10]");

IMPORTANT here we set the event ID variable. Each event in your tree should have a unique id with which to synchronise with weights.
    This is useful as it allows us to break the data up, perform seperate fits then combine the weights afterwards.
    The weights are not written into the input tree, rather they are stored seperately (as Weights) and combined when required. This allows you to use various sets of weights when performing fits or regenerate the weights after some corrections etc.

In [ ]:
splot.SetUp().SetIDBranchName("fgID")

### Create a signal PDF

The LoadSpecies Line adds the PDF Signal to the Extended Maximum Likelihood fit and creates a corresponding yield parameter Yld_Signal which will give the number of signal events in the data after the fit.
Note, I did not have to directly load this PDF I could have created more PDFs and combined them to make a more complicated PDF shape first.

We are just going to use a signal Gaussain with initial guess of mean 4 (with possible range 2-8), and sigma 1 (range 0.1-5)

In [ ]:
splot.SetUp().FactoryPDF("Gaussian::Signal( Mmiss, smean[6,4,7], swidth[0.2,0.0001,3] )");
splot.SetUp().LoadSpeciesPDF("Signal")

### Create a background PDF

Here we just use a quadratic Chebychev polynomial to describe a smooth background. You can add extra a2,a3,etc. to increase the degree of the polynomial

In [ ]:
splot.SetUp().FactoryPDF("Chebychev::BG(Mmiss,{a0[-0.1,-1,1],a1[0.1,-1,1]})");
splot.SetUp().LoadSpeciesPDF("BG",1)

Load the data giving the tree name then the file name.

In [ ]:
splot.LoadData("MyModel","Data.root")

Run the fit locally, this will also create the sWeights for each event. When it is finished it will display a plot of the signal and background fit to Mmiss and also show the residual and pull plots between the fit and data

There are different options for running. To just run standard use Here::Go and the fits will just be performed within this interactive session, sequentially in the case of bins.

Here::One(&RF,i) will run one singal fit from the different bins produce with LoadBinVars, where i gives the index of the bin

Use Proof::Go(&RF,N) and the fits will be run in parallel via PROOF with each split/bin running on a different core, where N = number of cores to use.

In [ ]:
ROOT.Here.Go(splot)

Now we will draw the resulting weighted phi distributions for the different polarisation states. First create canvases for drawing on.

In [ ]:
from ROOT import TCanvas
canvas = TCanvas("WeightedPlots","WeightedPlots")

And draw the weighted distributions using sPlot::DrawWeighted. The first string takes standard ROOT TTree::Draw arguments. The second is the name of the PDF corresponding to the species you want to draw.

In [ ]:
hPhi=ROOT.TH1F("hPhi","hPhi",100,-180,180);
splot.DrawWeighted("Phi>>hPhi","Signal");
splot.DrawWeighted("Phi","Signal","PolState==-1","same");
splot.DrawWeighted("Phi","Signal","PolState==1","same");
hPhi.SetMinimum(0)
canvas.Draw()

If we want to save the weighted tree with all branches inclduing weights you need to use the following line

In [ ]:
splot.DeleteWeightedTree()